# Llama 3 LoRA Fine-tuning on PokerBench Dataset

This notebook demonstrates how to fine-tune a language model on the RZ412/PokerBench dataset using LoRA (Low-Rank Adaptation).

## 1. Environment Setup and Dependencies

In [1]:
!pip install --upgrade transformers torch peft datasets bitsandbytes accelerate

  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.8.0-cp39-cp39-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached regex-2025.9.18-cp39-cp39-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.1.10-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.7 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached nvi

In [2]:
# Import necessary libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
from datasets import load_dataset, Dataset
import numpy as np
import pandas as pd
import json
import os
import warnings
from tqdm.auto import tqdm
import gc

# Suppress warnings
warnings.filterwarnings("ignore")

# Check device availability in priority order (optimized for GPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using NVIDIA GPU (CUDA): {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using Apple GPU (MPS)")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Optimized memory management
if device.type == "cuda":
    torch.cuda.empty_cache()
    # Set memory fraction for stability on Colab
    torch.cuda.set_per_process_memory_fraction(0.95)
    print("CUDA memory optimized for training")

elif device.type == "mps":
    torch.mps.empty_cache()
    gc.collect()
    print("MPS memory cleared")

Using NVIDIA GPU (CUDA): NVIDIA A100-SXM4-40GB
GPU Memory: 39.6 GB
CUDA memory optimized for training


## 2. Dataset Loading and Exploration

In [3]:
# Load the PokerBench dataset
print("Loading dataset...")

try:
    dataset = load_dataset("RZ412/PokerBench")
    print("Dataset loaded")
    print(f"Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

except Exception as e:
    print(f"Error loading dataset: {e}")

Loading dataset...


README.md: 0.00B [00:00, ?B/s]

postflop_500k_train_set_prompt_and_label(…):   0%|          | 0.00/561M [00:00<?, ?B/s]

preflop_60k_train_set_prompt_and_label.j(…):   0%|          | 0.00/59.2M [00:00<?, ?B/s]

postflop_10k_test_set_prompt_and_label.j(…):   0%|          | 0.00/11.2M [00:00<?, ?B/s]

(…)reflop_1k_test_set_prompt_and_label.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/563200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11000 [00:00<?, ? examples/s]

Dataset loaded
Train: 563200, Test: 11000


## 3. Data Preprocessing and Tokenization

In [5]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `CS6220` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `CS6220`


In [6]:
# Configuration for model and training
MODEL_NAME = "google/gemma-2b"

MAX_LENGTH = 512  # Reduced for memory efficiency
LORA_R = 8        # LoRA rank - lower = less memory
LORA_ALPHA = 32   # LoRA scaling parameter
LORA_DROPOUT = 0.1

print(f"Selected Model: {MODEL_NAME}")
print(f"Max Sequence Length: {MAX_LENGTH}")
print(f"LoRA Configuration: r={LORA_R}, alpha={LORA_ALPHA}, dropout={LORA_DROPOUT}")

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Add padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Added padding token (using EOS token)")

print(f"Tokenizer loaded. Vocab size: {len(tokenizer)}")

Selected Model: google/gemma-2b
Max Sequence Length: 512
LoRA Configuration: r=8, alpha=32, dropout=0.1
Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Tokenizer loaded. Vocab size: 256000


In [20]:
# Preprocess and format data for instruction following
def format_prompt(instruction, output):
    """Format instruction-output pairs for fine-tuning"""
    return f"### Instruction:\n{instruction}\n\n### Response:\n{output}"

def preprocess_function(examples):
    """Tokenize and format examples"""
    # Format prompts
    prompts = [
        format_prompt(inst, out)
        for inst, out in zip(examples['instruction'], examples['output'])
    ]

    # Tokenize with truncation and padding
    model_inputs = tokenizer(
        prompts,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length',  # Explicitly pad to max_length
        return_tensors=None         # Return lists for mapping
    )

    # For causal LM, labels are the same as input_ids
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs

# Process datasets
print("Preprocessing dataset...")

# Create balanced training set with both preflop and postflop examples
POSTFLOP_SAMPLES = 500  # Postflop examples from beginning (0-499)
PREFLOP_SAMPLES = 500   # Preflop examples from end (starting around row 10000+)
TOTAL_SUBSET = POSTFLOP_SAMPLES + PREFLOP_SAMPLES  # 1000 total samples
TRAIN_RATIO = 0.8    # 80% for training, 20% for validation

# Calculate split sizes
TRAIN_SUBSET = int(TOTAL_SUBSET * TRAIN_RATIO)  # 800 samples
VAL_SUBSET = TOTAL_SUBSET - TRAIN_SUBSET        # 200 samples

# Select balanced training data: postflop + preflop examples
postflop_train_data = dataset['train'].select(range(POSTFLOP_SAMPLES))  # First 500 (postflop)
preflop_start_idx = len(dataset['train']) - PREFLOP_SAMPLES  # Last 500 (preflop)
preflop_train_data = dataset['train'].select(range(preflop_start_idx, len(dataset['train'])))

# Combine postflop and preflop data
from datasets import concatenate_datasets
train_full = concatenate_datasets([postflop_train_data, preflop_train_data])
print(f"Combined training data: {len(postflop_train_data)} postflop + {len(preflop_train_data)} preflop = {len(train_full)} total")

# Shuffle the combined dataset before splitting (important for randomization)
train_full = train_full.shuffle(seed=42)
print("Dataset shuffled for randomized train/validation split")

# Split shuffled data into train/validation
train_dataset = train_full.select(range(TRAIN_SUBSET))
val_dataset = train_full.select(range(TRAIN_SUBSET, TOTAL_SUBSET))

# Keep test set completely separate for final evaluation only
test_dataset = dataset['test']  # Will be used only for final evaluation

print(f"Using {len(train_dataset)} training samples and {len(val_dataset)} validation samples (8:2 ratio)")
print(f"Test set reserved for final evaluation: {len(test_dataset)} samples")

# Apply preprocessing
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Preprocessing train dataset"
)

val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=val_dataset.column_names,
    desc="Preprocessing validation dataset"
)

# Note: test_dataset will be preprocessed later during final evaluation
print(f"Sample tokenized length: {len(train_dataset[0]['input_ids'])} tokens")
print("Dataset preprocessing complete!")

Preprocessing dataset...
Combined training data: 500 postflop + 500 preflop = 1000 total
Dataset shuffled for randomized train/validation split
Using 800 training samples and 200 validation samples (8:2 ratio)
Test set reserved for final evaluation: 11000 samples


Preprocessing train dataset:   0%|          | 0/800 [00:00<?, ? examples/s]

Preprocessing validation dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Sample tokenized length: 512 tokens
Dataset preprocessing complete!


## 4. Model Configuration

In [21]:
# Load model with memory optimization
print("Loading base model...")

# Model loading optimized for GPU
try:
    # Use optimal dtype based on device
    if device.type == "cuda":
        model_dtype = torch.float16  # GPU supports fp16
    else:
        model_dtype = torch.float32  # MPS/CPU fallback

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=model_dtype,
        device_map="auto" if device.type == "cuda" else None,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )

    # Move model to appropriate device with optimal precision
    if device.type == "mps":
        # MPS doesn't support all operations with float16, so use float32
        model = model.to(device, dtype=torch.float32)
        print(f"Model moved to MPS device with float32 precision")
    elif device.type == "cuda":
        model = model.to(device, dtype=torch.float16)
        print(f"Model moved to CUDA device with float16 precision")
    else:
        model = model.to(device)
        print(f"Model loaded on {device}")

    # Enable gradient checkpointing to save memory
    model.gradient_checkpointing_enable()
    print("Gradient checkpointing enabled")

    # Print model information
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Model Statistics:")
    print(f"  - Total parameters: {total_params:,}")
    print(f"  - Trainable parameters: {trainable_params:,}")
    print(f"  - Model size: ~{total_params * 4 / 1024**3:.2f} GB (float32)")

except Exception as e:
    print(f"Error loading model: {e}")
    print("Consider using a smaller model or CPU-only training")

Loading base model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model moved to CUDA device with float16 precision
Gradient checkpointing enabled
Model Statistics:
  - Total parameters: 2,506,172,416
  - Trainable parameters: 2,506,172,416
  - Model size: ~9.34 GB (float32)


In [ ]:
# Configure LoRA for efficient fine-tuning
print("Setting up LoRA...")

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj"
        # Note: "gate_proj", "up_proj", "down_proj" can be added for MLP layers if needed
    ],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA to the model
try:
    model = get_peft_model(model, lora_config)
    print("LoRA configuration applied")

    model.print_trainable_parameters()

    if device.type == "mps":
        torch.mps.empty_cache()
        gc.collect()

except Exception as e:
    print(f"Error applying LoRA: {e}")

Setting up LoRA...
LoRA configuration applied
trainable params: 1,843,200 || all params: 2,508,015,616 || trainable%: 0.0735


## 5. Training Configuration and Loop

In [33]:
# Training configuration optimized for Apple Silicon
output_dir = "./drive/MyDrive/poker-lora-model/" + MODEL_NAME.split("/")[-1] + "/"
os.makedirs(output_dir, exist_ok=True)

# Training arguments optimized for GPU with MPS compatibility
# Dynamic batch size based on device capabilities
if device.type == "cuda":
    batch_size = 2  # Larger batch size for GPU
    use_fp16 = True
    num_workers = 2
else:
    batch_size = 1  # Conservative for MPS/CPU
    use_fp16 = False
    num_workers = 0

training_args = TrainingArguments(
    output_dir=output_dir,

    # Training schedule
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=8 // batch_size,  # Maintain effective batch size

    # Optimization
    learning_rate=5e-5,
    warmup_steps=50,
    lr_scheduler_type="cosine",

    # Device-specific optimizations
    dataloader_num_workers=0,           # Disable workers to avoid tensor issues
    fp16=use_fp16,                      # Enable fp16 for GPU
    gradient_checkpointing=True,

    # Logging and saving - FIXED: More frequent monitoring
    logging_steps=5,                    # More frequent logging to catch issues
    save_steps=50,                      # More frequent saves
    save_total_limit=3,
    eval_strategy="steps",              # Fixed: was evaluation_strategy
    eval_steps=25,                      # More frequent evaluation

    # Other settings
    remove_unused_columns=False,
    push_to_hub=False,
    report_to=[],                       # Disable all logging including wandb

    # ADDED: Stability improvements
    max_grad_norm=1.0,                  # Gradient clipping to prevent exploding gradients
    weight_decay=0.01,                  # L2 regularization

    seed=42,                            # For reproducibility
)

print("Training configuration:")
print(f"  - Output directory: {output_dir}")
print(f"  - Learning rate: {training_args.learning_rate}")
print(f"  - Epochs: {training_args.num_train_epochs}")
print(f"  - Batch size: {training_args.per_device_train_batch_size}")
print(f"  - Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")

Training configuration:
  - Output directory: ./drive/MyDrive/poker-lora-model/gemma-2b/
  - Learning rate: 5e-05
  - Epochs: 3
  - Batch size: 2
  - Gradient accumulation: 4
  - Effective batch size: 8


In [24]:
# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
    pad_to_multiple_of=8,  # For efficiency
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Use validation set for training evaluation
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Trainer initialized successfully!")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset reserved for final evaluation: {len(test_dataset)}")

# Memory status before training
if device.type == "cuda":
    torch.cuda.empty_cache()
    print(f"GPU Memory before training: {torch.cuda.memory_allocated() / 1024**3:.2f} GB / {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
elif device.type == "mps":
    torch.mps.empty_cache()
    gc.collect()
    print("MPS cache cleared before training")

Trainer initialized successfully!
Training dataset size: 800
Validation dataset size: 200
Test dataset reserved for final evaluation: 11000
GPU Memory before training: 9.35 GB / 39.6 GB


In [25]:
# Start training
print("Starting LoRA fine-tuning...")

try:
    # Start training
    training_result = trainer.train()

    print("\nTraining completed successfully!")
    print(f"Final training loss: {training_result.training_loss:.4f}")

    # Save training metrics
    trainer.log_metrics("train", training_result.metrics)
    trainer.save_metrics("train", training_result.metrics)

except Exception as e:
    print(f"Training error: {e}")

    # Clear memory on error
    if device.type == "mps":
        torch.mps.empty_cache()
    elif device.type == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Starting LoRA fine-tuning...


Step,Training Loss,Validation Loss
25,2.259800,2.201604
50,1.621700,1.502348
75,0.571400,0.514940
100,0.337100,0.338494
125,0.304600,0.304868
150,0.289000,0.280933
175,0.268100,0.266293
200,0.266900,0.261862
225,0.262500,0.258886
250,0.251100,0.256857



Training completed successfully!
Final training loss: 0.6473
***** train metrics *****
  epoch                    =        3.0
  total_flos               = 13621176GF
  train_loss               =     0.6473
  train_runtime            = 0:04:59.61
  train_samples_per_second =       8.01
  train_steps_per_second   =      1.001


## 6. Model Evaluation and Testing

In [26]:
# Evaluate the model
print("Evaluating the fine-tuned model...")

try:
    eval_results = trainer.evaluate()

    print("\nEvaluation Results:")
    for key, value in eval_results.items():
        print(f"  - {key}: {value:.4f}")

    # Save evaluation metrics
    trainer.save_metrics("eval", eval_results)

except Exception as e:
    print(f"Evaluation error: {e}")

# Test inference with sample poker scenarios
def generate_response(instruction, max_length=100):
    """Generate a response for a poker instruction"""

    # Format the prompt
    prompt = format_prompt(instruction, "")

    # Tokenize
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the generated part
    generated_part = response[len(prompt):].strip()

    return generated_part

print("\nTesting model with sample poker scenarios...")

Evaluating the fine-tuned model...



Evaluation Results:
  - eval_loss: 0.2558
  - eval_runtime: 6.0916
  - eval_samples_per_second: 32.8320
  - eval_steps_per_second: 16.4160
  - epoch: 3.0000

Testing model with sample poker scenarios...


In [27]:
# Preflop vs Postflop Performance Analysis
print("\n" + "="*60)
print("PREFLOP VS POSTFLOP PERFORMANCE ANALYSIS")
print("="*60)

def extract_action_and_amount(output_text):
    """Extract action and amount from model output"""
    output_text = output_text.strip().lower()

    # Split by spaces and get first tokens
    tokens = output_text.split()
    if not tokens:
        return None, None

    action = tokens[0]
    amount = None

    # For actions that include amounts (bet, raise, call)
    if action in ['bet', 'raise', 'call'] and len(tokens) > 1:
        # Try to extract numeric amount from second token
        try:
            amount = float(tokens[1])
        except (ValueError, IndexError):
            amount = None

    return action, amount

def analyze_preflop_postflop(model, tokenizer, preflop_dataset, postflop_dataset):
    """Analyze performance difference between preflop and postflop scenarios"""
    model_device = next(model.parameters()).device

    preflop_stats = {
        'total': 0, 'action_correct': 0, 'exact_match_correct': 0,
        'actions': {'fold': 0, 'call': 0, 'bet': 0, 'raise': 0, 'check': 0}
    }
    postflop_stats = {
        'total': 0, 'action_correct': 0, 'exact_match_correct': 0,
        'actions': {'fold': 0, 'call': 0, 'bet': 0, 'raise': 0, 'check': 0}
    }

    # Process preflop examples
    print(f"Evaluating {len(preflop_dataset)} preflop examples...")
    for i, example in enumerate(preflop_dataset):
        if i % 100 == 0:
            print(f"Processing preflop sample {i+1}/{len(preflop_dataset)}...")

        stats = preflop_stats
        stats['total'] += 1

        # Format prompt and generate response
        prompt = format_prompt(example['instruction'], "")
        prompt = prompt.replace("### Response:\n", "### Response:\n")

        try:
            inputs = tokenizer.encode(prompt, return_tensors="pt").to(model_device)

            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 50,
                    temperature=0.3,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )

            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated_part = response[len(prompt):].strip()

            # Extract actions and amounts
            expected_action, expected_amount = extract_action_and_amount(example['output'])
            generated_action, generated_amount = extract_action_and_amount(generated_part)

            if expected_action is None:
                continue

            # Count action types
            if expected_action in stats['actions']:
                stats['actions'][expected_action] += 1

            # Check accuracy
            if expected_action == generated_action:
                stats['action_correct'] += 1

                # Check exact match for actions with amounts
                if expected_action in ['bet', 'raise', 'call']:
                    if expected_amount is not None and generated_amount is not None:
                        if abs(expected_amount - generated_amount) < 0.01:
                            stats['exact_match_correct'] += 1
                    elif expected_amount is None and generated_amount is None:
                        stats['exact_match_correct'] += 1
                else:
                    stats['exact_match_correct'] += 1

        except Exception as e:
            continue

    # Process postflop examples
    print(f"Evaluating {len(postflop_dataset)} postflop examples...")
    for i, example in enumerate(postflop_dataset):
        if i % 100 == 0:
            print(f"Processing postflop sample {i+1}/{len(postflop_dataset)}...")

        stats = postflop_stats
        stats['total'] += 1

        # Format prompt and generate response
        prompt = format_prompt(example['instruction'], "")
        prompt = prompt.replace("### Response:\n", "### Response:\n")

        try:
            inputs = tokenizer.encode(prompt, return_tensors="pt").to(model_device)

            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 50,
                    temperature=0.3,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=tokenizer.eos_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )

            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated_part = response[len(prompt):].strip()

            # Extract actions and amounts
            expected_action, expected_amount = extract_action_and_amount(example['output'])
            generated_action, generated_amount = extract_action_and_amount(generated_part)

            if expected_action is None:
                continue

            # Count action types
            if expected_action in stats['actions']:
                stats['actions'][expected_action] += 1

            # Check accuracy
            if expected_action == generated_action:
                stats['action_correct'] += 1

                # Check exact match for actions with amounts
                if expected_action in ['bet', 'raise', 'call']:
                    if expected_amount is not None and generated_amount is not None:
                        if abs(expected_amount - generated_amount) < 0.01:
                            stats['exact_match_correct'] += 1
                    elif expected_amount is None and generated_amount is None:
                        stats['exact_match_correct'] += 1
                else:
                    stats['exact_match_correct'] += 1

        except Exception as e:
            continue

    return preflop_stats, postflop_stats

# Create separate preflop and postflop datasets
# Dataset structure: 0-9999 are postflop, 10001-11000 are preflop
postflop_test_data = dataset['test'].select(range(0, 500))  # 500 postflop examples (from 0-9999)
preflop_test_data = dataset['test'].select(range(10001, min(10501, len(dataset['test']))))  # 500 preflop examples (from 10001-11000)

print(f"Using {len(postflop_test_data)} postflop and {len(preflop_test_data)} preflop examples for evaluation")

# Run preflop vs postflop analysis
preflop_stats, postflop_stats = analyze_preflop_postflop(model, tokenizer, preflop_test_data, postflop_test_data)

# Calculate and display results
def display_scenario_stats(stats, scenario_name):
    if stats['total'] == 0:
        print(f"\n{scenario_name}: No samples found")
        return

    aa = (stats['action_correct'] / stats['total']) * 100
    em = (stats['exact_match_correct'] / stats['total']) * 100

    print(f"\n{scenario_name} Performance:")
    print(f"  Total Samples: {stats['total']}")
    print(f"  Action Accuracy: {aa:.2f}% ({stats['action_correct']}/{stats['total']})")
    print(f"  Exact Match Accuracy: {em:.2f}% ({stats['exact_match_correct']}/{stats['total']})")

    print(f"  Action Distribution:")
    for action, count in stats['actions'].items():
        if count > 0:
            percentage = (count / stats['total']) * 100
            print(f"    {action.upper()}: {count} ({percentage:.1f}%)")

display_scenario_stats(preflop_stats, "PREFLOP")
display_scenario_stats(postflop_stats, "POSTFLOP")

# Overall assessment
if preflop_stats['total'] > 0 and postflop_stats['total'] > 0:
    preflop_aa = (preflop_stats['action_correct'] / preflop_stats['total']) * 100
    postflop_aa = (postflop_stats['action_correct'] / postflop_stats['total']) * 100

    print(f"\n" + "-"*50)
    print("OVERALL ASSESSMENT")
    print("-"*50)
    print(f"Preflop accuracy: {preflop_aa:.1f}%")
    print(f"Postflop accuracy: {postflop_aa:.1f}%")

print("\n" + "="*60)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



PREFLOP VS POSTFLOP PERFORMANCE ANALYSIS
Using 500 postflop and 500 preflop examples for evaluation
Evaluating 500 preflop examples...
Processing preflop sample 1/500...
Processing preflop sample 101/500...
Processing preflop sample 201/500...
Processing preflop sample 301/500...
Processing preflop sample 401/500...
Evaluating 500 postflop examples...
Processing postflop sample 1/500...
Processing postflop sample 101/500...
Processing postflop sample 201/500...
Processing postflop sample 301/500...
Processing postflop sample 401/500...

PREFLOP Performance:
  Total Samples: 500
  Action Accuracy: 32.00% (160/500)
  Exact Match Accuracy: 24.00% (120/500)
  Action Distribution:
    FOLD: 140 (28.0%)
    CALL: 114 (22.8%)
    RAISE: 125 (25.0%)
    CHECK: 121 (24.2%)

POSTFLOP Performance:
  Total Samples: 500
  Action Accuracy: 40.60% (203/500)
  Exact Match Accuracy: 38.60% (193/500)
  Action Distribution:
    FOLD: 129 (25.8%)
    CALL: 127 (25.4%)
    BET: 58 (11.6%)
    RAISE: 80 (1

## 7. Model Saving and Export

In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Save the fine-tuned LoRA model
print("Saving the fine-tuned model...")

try:
    # Save the model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print(f"Model saved to: {output_dir}")

    # List saved files
    saved_files = os.listdir(output_dir)
    print(f"Saved files: {saved_files}")

    # Calculate model size
    total_size = sum(
        os.path.getsize(os.path.join(output_dir, f))
        for f in saved_files
        if os.path.isfile(os.path.join(output_dir, f))
    )
    print(f"Total model size: {total_size / (1024**2):.2f} MB")

except Exception as e:
    print(f"Error saving model: {e}")

print("\nFine-tuning process completed successfully!")
print(f"The fine-tuned model is ready at: {os.path.abspath(output_dir)}")

Saving the fine-tuned model...
Model saved to: ./drive/MyDrive/poker-lora-model/gemma-2b/
Saved files: ['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.model', 'tokenizer.json']
Total model size: 43.90 MB

Fine-tuning process completed successfully!
Your fine-tuned model is ready at: /content/drive/MyDrive/poker-lora-model/gemma-2b


# Meta-Llama-3-8B Fine-tuning Setup

## Setup Requirements for Llama-3-8B

In [35]:
# Llama-3-8B Configuration
LLAMA_MODEL_NAME = "meta-llama/Meta-Llama-3-8B"
LLAMA_MAX_LENGTH = 512
LLAMA_LORA_R = 16
LLAMA_LORA_ALPHA = 32
LLAMA_LORA_DROPOUT = 0.1

# Dataset configuration for Llama-3 (8:2 train/validation ratio)
LLAMA_POSTFLOP_SAMPLES = 1250  # Postflop examples from beginning
LLAMA_PREFLOP_SAMPLES = 1250   # Preflop examples from end
LLAMA_TOTAL_SUBSET = LLAMA_POSTFLOP_SAMPLES + LLAMA_PREFLOP_SAMPLES  # 2500 total samples
LLAMA_TRAIN_RATIO = 0.8    # 80% for training, 20% for validation
LLAMA_TRAIN_SUBSET = int(LLAMA_TOTAL_SUBSET * LLAMA_TRAIN_RATIO)  # 2000 samples
LLAMA_VAL_SUBSET = LLAMA_TOTAL_SUBSET - LLAMA_TRAIN_SUBSET        # 500 samples

print(f"Model: {LLAMA_MODEL_NAME}")
print(f"LoRA Configuration: r={LLAMA_LORA_R}, alpha={LLAMA_LORA_ALPHA}, dropout={LLAMA_LORA_DROPOUT}")
print(f"Dataset Configuration:")
print(f"  - Total samples: {LLAMA_TOTAL_SUBSET}")
print(f"  - Training samples: {LLAMA_TRAIN_SUBSET} (80%)")
print(f"  - Validation samples: {LLAMA_VAL_SUBSET} (20%)")

Model: meta-llama/Meta-Llama-3-8B
LoRA Configuration: r=16, alpha=32, dropout=0.1
Dataset Configuration:
  - Total samples: 2500
  - Training samples: 2000 (80%)
  - Validation samples: 500 (20%)


In [39]:
# Load Llama-3 Tokenizer
print("Loading Llama-3-8B tokenizer...")

try:
    llama_tokenizer = AutoTokenizer.from_pretrained(
        LLAMA_MODEL_NAME,
        token=True,
        trust_remote_code=True
    )

    if llama_tokenizer.pad_token is None:
        llama_tokenizer.pad_token = llama_tokenizer.eos_token
        print("Padding token configured")

    print(f"Llama-3 tokenizer loaded")
    print(f"Vocabulary size: {len(llama_tokenizer):,}")
    print(f"Model max length: {llama_tokenizer.model_max_length}")

except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Make sure you have HF token configured")

Loading Llama-3-8B tokenizer...


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Padding token configured
Llama-3 tokenizer loaded
Vocabulary size: 128,256
Model max length: 1000000000000000019884624838656


In [40]:
# Load Llama-3-8B Model with 4-bit Quantization
print("Loading Llama-3-8B model with quantization...")

# Configure 4-bit quantization for memory efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

try:
    llama_model = AutoModelForCausalLM.from_pretrained(
        LLAMA_MODEL_NAME,
        quantization_config=quantization_config,
        device_map="auto",  # Automatically distribute across available devices
        torch_dtype=torch.float16,
        token=True,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )

    print("Llama-3-8B model loaded with 4-bit quantization")

    total_params = sum(p.numel() for p in llama_model.parameters())
    print(f"Total parameters: {total_params:,}")
    print(f"Model device: {next(llama_model.parameters()).device}")

    llama_model.gradient_checkpointing_enable()
    print("Gradient checkpointing enabled")

    llama_model = prepare_model_for_kbit_training(llama_model)
    print("Model prepared for 4-bit training")

except Exception as e:
    print(f"Error loading model: {e}")
    print("This might be due to:")
    print("1. Insufficient GPU memory (need ~8GB for quantized model)")
    print("2. Missing authentication or access permissions")
    print("3. Missing required packages (bitsandbytes, accelerate)")

Loading Llama-3-8B model with quantization...


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

Llama-3-8B model loaded with 4-bit quantization
Total parameters: 4,540,600,320
Model device: cuda:0
Gradient checkpointing enabled
Model prepared for 4-bit training


In [41]:
# Configure LoRA for Llama-3-8B
print("Setting up LoRA configuration for Llama-3...")

llama_lora_config = LoraConfig(
    r=LLAMA_LORA_R,
    lora_alpha=LLAMA_LORA_ALPHA,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj"  # Attention layers
        # Note: "gate_proj", "up_proj", "down_proj" (MLP layers) can be added if needed
    ],
    lora_dropout=LLAMA_LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

try:
    llama_model = get_peft_model(llama_model, llama_lora_config)
    print("LoRA configuration applied")

    llama_model.print_trainable_parameters()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()

except Exception as e:
    print(f"Error applying LoRA: {e}")

Setting up LoRA configuration for Llama-3...
LoRA configuration applied
trainable params: 13,631,488 || all params: 8,043,892,736 || trainable%: 0.1695


In [42]:
# Prepare Dataset for Llama-3 Training
print("Preparing dataset for Llama-3 training...")

# Create balanced Llama-3 training set with both preflop and postflop examples
llama_postflop_train_data = dataset['train'].select(range(LLAMA_POSTFLOP_SAMPLES))  # First 1250 (postflop)
llama_preflop_start_idx = len(dataset['train']) - LLAMA_PREFLOP_SAMPLES  # Last 1250 (preflop)
llama_preflop_train_data = dataset['train'].select(range(llama_preflop_start_idx, len(dataset['train'])))

# Combine postflop and preflop data for Llama-3
llama_train_full = concatenate_datasets([llama_postflop_train_data, llama_preflop_train_data])
print(f"Llama-3 combined training data: {len(llama_postflop_train_data)} postflop + {len(llama_preflop_train_data)} preflop = {len(llama_train_full)} total")

# Shuffle the combined dataset before splitting (important for randomization)
llama_train_full = llama_train_full.shuffle(seed=42)
print("Llama-3 dataset shuffled for randomized train/validation split")

# Split shuffled data into train/validation
llama_train_dataset = llama_train_full.select(range(LLAMA_TRAIN_SUBSET))
llama_val_dataset = llama_train_full.select(range(LLAMA_TRAIN_SUBSET, LLAMA_TOTAL_SUBSET))

# Test set remains the same for final evaluation
llama_test_dataset = dataset['test']

print(f"Llama-3 training samples: {len(llama_train_dataset)} (80%)")
print(f"Llama-3 validation samples: {len(llama_val_dataset)} (20%)")
print(f"Llama-3 test set reserved for final evaluation: {len(llama_test_dataset)}")

# Preprocess with Llama tokenizer (reuses existing format_prompt function)
def llama_preprocess_function(examples):
    """Tokenize examples for Llama-3 - reuses format_prompt function"""
    prompts = [
        format_prompt(inst, out)
        for inst, out in zip(examples['instruction'], examples['output'])
    ]

    model_inputs = llama_tokenizer(
        prompts,
        max_length=LLAMA_MAX_LENGTH,
        truncation=True,
        padding=False,
        return_tensors=None
    )

    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Apply preprocessing
print("Preprocessing Llama-3 datasets...")

llama_train_dataset = llama_train_dataset.map(
    llama_preprocess_function,
    batched=True,
    remove_columns=llama_train_dataset.column_names,
    desc="Preprocessing Llama train dataset"
)

llama_val_dataset = llama_val_dataset.map(
    llama_preprocess_function,
    batched=True,
    remove_columns=llama_val_dataset.column_names,
    desc="Preprocessing Llama validation dataset"
)

print("Dataset preprocessing complete")
print(f"Sample tokenized length: {len(llama_train_dataset[0]['input_ids'])} tokens")

Preparing dataset for Llama-3 training...
Llama-3 combined training data: 1250 postflop + 1250 preflop = 2500 total
Llama-3 dataset shuffled for randomized train/validation split
Llama-3 training samples: 2000 (80%)
Llama-3 validation samples: 500 (20%)
Llama-3 test set reserved for final evaluation: 11000
Preprocessing Llama-3 datasets...


Preprocessing Llama train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Preprocessing Llama validation dataset:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset preprocessing complete
Sample tokenized length: 284 tokens


In [43]:
# Setup Training Configuration for Llama-3
print("Configuring training for Llama-3...")

# Output directory for Llama-3 model
llama_output_dir = llama_output_dir = "./drive/MyDrive/poker-lora-model/Meta-Llama-3-8B/"
os.makedirs(llama_output_dir, exist_ok=True)

# Optimized training arguments for Llama-3 (GPU-first)
if device.type == "cuda":
    llama_batch_size = 1  # Conservative for 8B model
    llama_use_fp16 = True
    llama_num_workers = 2
else:
    llama_batch_size = 1
    llama_use_fp16 = False
    llama_num_workers = 0

llama_training_args = TrainingArguments(
    output_dir=llama_output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=llama_batch_size,
    per_device_eval_batch_size=llama_batch_size,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_steps=100,
    lr_scheduler_type="cosine",
    dataloader_num_workers=llama_num_workers,
    fp16=llama_use_fp16,
    gradient_checkpointing=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=50,

    # Other settings
    remove_unused_columns=False,
    push_to_hub=False,
    report_to=None,

    # Stability improvements
    max_grad_norm=1.0,
    weight_decay=0.01,
    seed=42,
)


print(f"Output directory: {llama_output_dir}")
print(f"Effective batch size: {llama_training_args.per_device_train_batch_size * llama_training_args.gradient_accumulation_steps}")
print("Training configuration ready")

# Data collator for Llama-3
llama_data_collator = DataCollatorForLanguageModeling(
    tokenizer=llama_tokenizer,
    mlm=False,
    pad_to_multiple_of=8,
)

Configuring training for Llama-3...
Output directory: ./poker-lora-llama3-8b/
Effective batch size: 8
Training configuration ready


In [44]:
# Initialize Trainer and Start Training
print("Initializing Llama-3 trainer...")

# Create trainer for Llama-3
llama_trainer = Trainer(
    model=llama_model,
    args=llama_training_args,
    train_dataset=llama_train_dataset,
    eval_dataset=llama_val_dataset,  # Use validation set for training evaluation
    data_collator=llama_data_collator,
    tokenizer=llama_tokenizer,
)

print("Llama-3 trainer initialized")
print(f"Training dataset size: {len(llama_train_dataset)}")
print(f"Validation dataset size: {len(llama_val_dataset)}")
print(f"Test dataset reserved for final evaluation: {len(llama_test_dataset)}")

# Pre-training memory management
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU Memory before Llama training: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
elif torch.backends.mps.is_available():
    torch.mps.empty_cache()
    print("MPS cache cleared")

print(f"Training with batch_size={llama_batch_size}, fp16={llama_use_fp16}")
print("Ready to start Llama-3 training")

Initializing Llama-3 trainer...
Llama-3 trainer initialized
Training dataset size: 2000
Validation dataset size: 500
Test dataset reserved for final evaluation: 11000
GPU Memory before Llama training: 16.70 GB
Training with batch_size=1, fp16=True
Ready to start Llama-3 training


In [45]:
# Start Llama-3 Training
print("Starting Llama-3-8B fine-tuning...")

try:
    # Start training
    llama_training_result = llama_trainer.train()

    print("\n" + "="*60)
    print("LLAMA-3 TRAINING COMPLETED SUCCESSFULLY!")
    print("Training completed")
    print(f"Final training loss: {llama_training_result.training_loss:.4f}")

    llama_trainer.log_metrics("train", llama_training_result.metrics)
    llama_trainer.save_metrics("train", llama_training_result.metrics)

    print("Training metrics saved")

except Exception as e:
    print(f"Training error: {e}")
    print("\nTroubleshooting tips:")
    print("1. Check GPU memory usage")
    print("2. Reduce batch size or model parameters if needed")
    print("3. Ensure sufficient disk space for checkpoints")

    # Clear memory on error
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    elif torch.backends.mps.is_available():
        torch.mps.empty_cache()
    gc.collect()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


Starting Llama-3-8B fine-tuning...


Step,Training Loss,Validation Loss
50,0.266200,0.218581
100,0.160900,0.165863
150,0.118400,0.120034
200,0.111700,0.114691


Step,Training Loss,Validation Loss
50,0.266200,0.218581
100,0.160900,0.165863
150,0.118400,0.120034
200,0.111700,0.114691
250,0.103000,0.105013
300,0.099800,0.099538
350,0.092900,0.094207
400,0.088900,0.092010
450,0.087800,0.089412
500,0.081800,0.084988



LLAMA-3 TRAINING COMPLETED SUCCESSFULLY!
Training completed
Final training loss: 0.1760
***** train metrics *****
  epoch                    =        3.0
  total_flos               = 63962586GF
  train_loss               =      0.176
  train_runtime            = 1:11:44.02
  train_samples_per_second =      1.394
  train_steps_per_second   =      0.174
Training metrics saved


In [46]:
# Evaluate Llama-3 Model Performance
print("Evaluating Llama-3 performance...")

# Create separate preflop and postflop datasets for Llama-3
# Dataset structure: 0-9999 are postflop, 10001-11000 are preflop
llama_postflop_test_data = dataset['test'].select(range(0, 500))  # 500 postflop examples (from 0-9999)
llama_preflop_test_data = dataset['test'].select(range(10001, min(10501, len(dataset['test']))))  # 500 preflop examples (from 10001-11000)

print(f"Using {len(llama_postflop_test_data)} postflop and {len(llama_preflop_test_data)} preflop examples for Llama-3 evaluation")

# Use the existing evaluation function with Llama-3 model and datasets
print("Starting Llama-3 evaluation...")
llama_preflop_stats, llama_postflop_stats = analyze_preflop_postflop(
    llama_model, llama_tokenizer, llama_preflop_test_data, llama_postflop_test_data
)

Evaluating Llama-3 performance...
Using 500 postflop and 500 preflop examples for Llama-3 evaluation
Starting Llama-3 evaluation...
Evaluating 500 preflop examples...
Processing preflop sample 1/500...
Processing preflop sample 101/500...
Processing preflop sample 201/500...
Processing preflop sample 301/500...
Processing preflop sample 401/500...
Evaluating 500 postflop examples...
Processing postflop sample 1/500...
Processing postflop sample 101/500...
Processing postflop sample 201/500...
Processing postflop sample 301/500...
Processing postflop sample 401/500...


In [47]:
# Display Llama-3 results
display_scenario_stats(llama_preflop_stats, "LLAMA-3 PREFLOP")
display_scenario_stats(llama_postflop_stats, "LLAMA-3 POSTFLOP")

# Overall Llama-3 assessment
if llama_preflop_stats['total'] > 0 and llama_postflop_stats['total'] > 0:
    preflop_aa = (llama_preflop_stats['action_correct'] / llama_preflop_stats['total']) * 100
    postflop_aa = (llama_postflop_stats['action_correct'] / llama_postflop_stats['total']) * 100

    print("\nLlama-3 Overall Assessment:")
    print("Fine-tuning completed successfully")
    print(f"Preflop accuracy: {preflop_aa:.1f}%")
    print(f"Postflop accuracy: {postflop_aa:.1f}%")


LLAMA-3 PREFLOP Performance:
  Total Samples: 500
  Action Accuracy: 68.80% (344/500)
  Exact Match Accuracy: 59.80% (299/500)
  Action Distribution:
    FOLD: 140 (28.0%)
    CALL: 114 (22.8%)
    RAISE: 125 (25.0%)
    CHECK: 121 (24.2%)

LLAMA-3 POSTFLOP Performance:
  Total Samples: 500
  Action Accuracy: 52.80% (264/500)
  Exact Match Accuracy: 48.00% (240/500)
  Action Distribution:
    FOLD: 129 (25.8%)
    CALL: 127 (25.4%)
    BET: 58 (11.6%)
    RAISE: 80 (16.0%)
    CHECK: 106 (21.2%)

Llama-3 Overall Assessment:
Fine-tuning completed successfully
Preflop accuracy: 68.8%
Postflop accuracy: 52.8%


In [48]:
# Save Llama-3 Model
print("Saving Llama-3 fine-tuned model...")
llama_output_dir = "./drive/MyDrive/poker-lora-model/Meta-Llama-3-8B/"

try:
    # Save the model and tokenizer
    llama_model.save_pretrained(llama_output_dir)
    llama_tokenizer.save_pretrained(llama_output_dir)

    print(f"Llama-3 model saved to: {llama_output_dir}")

    # List saved files
    saved_files = os.listdir(llama_output_dir)
    print(f"Saved files: {saved_files}")

    # Calculate model size
    total_size = sum(
        os.path.getsize(os.path.join(llama_output_dir, f))
        for f in saved_files
        if os.path.isfile(os.path.join(llama_output_dir, f))
    )
    print(f"Total Llama-3 model size: {total_size / (1024**2):.2f} MB")

except Exception as e:
    print(f"Error saving Llama-3 model: {e}")

print("\nLlama-3-8B fine-tuning completed")
print(f"Fine-tuned model ready at: {os.path.abspath(llama_output_dir)}")
print("Model ready for poker decision making")

Saving Llama-3 fine-tuned model...
Llama-3 model saved to: ./drive/MyDrive/poker-lora-model/Meta-Llama-3-8B/
Saved files: ['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'tokenizer.json']
Total Llama-3 model size: 68.50 MB

Llama-3-8B fine-tuning completed
Fine-tuned model ready at: /content/drive/MyDrive/poker-lora-model/Meta-Llama-3-8B
Model ready for poker decision making
